In [53]:
import pandas as pd
import sys
sys.path.append('../../ResponsibleAIToolbox-Mitigation/')
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from errorsmitigation.dataprocessing import DataRebalance
from databalanceanalysis.databalanceanalysis.utils import undummify

In [54]:
data_dir = '../datasets/hr_promotion'
df =  pd.read_csv(data_dir + '/train.csv')
cols_of_interest = ['education']
label_col = 'is_promoted'
seed = 42
df.shape
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [55]:
df['education'] = df['education'].fillna("Unknown")

In [56]:
from databalanceanalysis.databalanceanalysis.feature_measures import FeatureBalanceMeasure
feature_measures = FeatureBalanceMeasure( cols_of_interest, label_col)

feat_measures = feature_measures.measures(df)
feat_measures

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Master's & above,Bachelor's,education,0.016528,0.183561,0.002310,0.002694,5.266922,-0.715333,0.288878,0.395743
1,Master's & above,Unknown,education,0.047916,0.665851,0.057839,0.063631,-10.073010,2.489677,-0.923880,0.211845
2,Master's & above,Below Secondary,education,0.015330,0.169054,0.062836,0.068778,-20.881418,3.089005,-1.965732,0.072022
3,Bachelor's,Unknown,education,0.031388,0.482290,0.055529,0.060937,-15.339932,3.205010,-1.212758,0.156005
4,Bachelor's,Below Secondary,education,-0.001199,-0.014507,0.060526,0.066085,-26.148341,3.804338,-2.254610,0.054735
5,Unknown,Below Secondary,education,-0.032586,-0.496796,0.004997,0.005148,-10.808408,0.599328,-1.041852,0.187028


In [57]:
# OneHotEncoder for categorical features

target = df['education']
df = df.drop(["education"], axis = 1)
df = pd.get_dummies(df, drop_first=False, prefix_sep="-")
# handle null values
df.dropna(inplace=True)

df.shape
df['education'] = target

In [58]:
smote_tomek = SMOTETomek()
smote = SMOTE()
data_rebalance_smote =  DataRebalance(df, 'education', None, None, None, smote, None)
data_rebalance_tomek = DataRebalance(df, 'education', None, None, None, None, None)
data_rebalance_smote_tomek = DataRebalance(df, 'education', None, None, None, None, None)
smote_df = data_rebalance_smote.Rebalance()
tomek_df = data_rebalance_tomek.Rebalance()
smote_tomek_df = data_rebalance_smote_tomek.Rebalance()


In [59]:
smote_df

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_Analytics,...,region_region_6,region_region_7,region_region_8,region_region_9,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing,education
0,65438,1,35,5.000000,8,1,0,49,0,0,...,0,1,0,0,1,0,0,0,1,Master's & above
1,65141,1,30,5.000000,4,0,0,60,0,0,...,0,0,0,0,0,1,1,0,0,Bachelor's
2,7513,1,34,3.000000,7,0,0,50,0,0,...,0,0,0,0,0,1,0,0,1,Bachelor's
3,2542,2,39,1.000000,10,0,0,50,0,0,...,0,0,0,0,0,1,1,0,0,Bachelor's
4,48945,1,45,3.000000,2,0,0,73,0,0,...,0,0,0,0,0,1,1,0,0,Bachelor's
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133611,61894,1,46,4.761019,4,0,0,53,0,0,...,0,0,0,0,0,1,0,0,0,Unknown
133612,37798,1,34,3.000000,8,0,0,49,0,0,...,0,0,0,0,0,1,0,0,0,Unknown
133613,55651,1,28,4.432024,3,1,0,83,0,1,...,0,0,0,0,0,1,0,0,1,Unknown
133614,42674,1,30,1.145528,4,0,0,48,0,0,...,0,0,0,0,0,1,0,0,1,Unknown


In [60]:
df = undummify(df, "-")
smote_df = undummify(smote_df, "-")
smote_tomek_df = undummify(smote_tomek_df, "-")
tomek_df = undummify(tomek_df, "-")

In [62]:
smote_df

,employee,no,age,previous,length,KPIs,awards,avg,is,department,region,gender,recruitment,education
0,id,of_trainings,35,year_rating,of_service,met >80%,won?,training_score,promoted,Sales & Marketing,region_7,f,channel_sourcing,Master's & above
1,id,of_trainings,30,year_rating,of_service,met >80%,won?,training_score,promoted,Operations,region_22,m,channel_other,Bachelor's
2,id,of_trainings,34,year_rating,of_service,met >80%,won?,training_score,promoted,Sales & Marketing,region_19,m,channel_sourcing,Bachelor's
3,id,of_trainings,39,year_rating,of_service,met >80%,won?,training_score,promoted,Sales & Marketing,region_23,m,channel_other,Bachelor's
4,id,of_trainings,45,year_rating,of_service,met >80%,won?,training_score,promoted,Technology,region_26,m,channel_other,Bachelor's
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133611,id,of_trainings,46,year_rating,of_service,met >80%,won?,training_score,promoted,Sales & Marketing,region_1,m,channel_other,Unknown
133612,id,of_trainings,34,year_rating,of_service,met >80%,won?,training_score,promoted,Analytics,region_1,m,channel_other,Unknown
133613,id,of_trainings,28,year_rating,of_service,met >80%,won?,training_score,promoted,Analytics,region_1,m,channel_sourcing,Unknown
133614,id,of_trainings,30,year_rating,of_service,met >80%,won?,training_score,promoted,Sales & Marketing,region_1,m,channel_sourcing,Unknown


In [61]:
from databalanceanalysis.databalanceanalysis.feature_measures import FeatureBalanceMeasure
feat_measures = feature_measures.measures(smote_df)

KeyError: 'is_promoted'

In [ ]:
feat_measures

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Master's & above,Bachelor's,education,-0.032032,-0.496777,-0.027991,-0.031517,-0.198215,-0.496777,0.0,0.5
1,Master's & above,Unknown,education,0.044665,2.274715,0.039030,0.040989,0.276389,2.274715,0.0,0.5
2,Master's & above,Below Secondary,education,0.042121,1.871201,0.036806,0.038740,0.260643,1.871201,0.0,0.5
3,Bachelor's,Unknown,education,0.076697,2.771492,0.067021,0.072506,0.474604,2.771492,0.0,0.5
4,Bachelor's,Below Secondary,education,0.074153,2.367978,0.064797,0.070257,0.458858,2.367978,0.0,0.5
5,Unknown,Below Secondary,education,-0.002545,-0.403514,-0.002224,-0.002249,-0.015746,-0.403514,0.0,0.5


In [ ]:
feature_measures.measures(smote_df)

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Master's & above,Bachelor's,education,-0.032032,-0.496777,-0.027991,-0.031517,-0.198215,-0.496777,0.0,0.5
1,Master's & above,Unknown,education,0.044665,2.274715,0.039030,0.040989,0.276389,2.274715,0.0,0.5
2,Master's & above,Below Secondary,education,0.042121,1.871201,0.036806,0.038740,0.260643,1.871201,0.0,0.5
3,Bachelor's,Unknown,education,0.076697,2.771492,0.067021,0.072506,0.474604,2.771492,0.0,0.5
4,Bachelor's,Below Secondary,education,0.074153,2.367978,0.064797,0.070257,0.458858,2.367978,0.0,0.5
5,Unknown,Below Secondary,education,-0.002545,-0.403514,-0.002224,-0.002249,-0.015746,-0.403514,0.0,0.5


In [ ]:
feature_measures.measures(smote_tomek_df)

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Master's & above,Bachelor's,education,-0.036868,-0.534748,-0.030852,-0.035003,-1.243835,-0.398282,-0.033054,0.487854
1,Master's & above,Unknown,education,0.045782,2.103520,0.040138,0.042289,-0.651863,2.224432,-0.029216,0.489264
2,Master's & above,Below Secondary,education,0.045624,2.078929,0.039802,0.041948,1.556743,1.915696,0.037824,0.486102
3,Bachelor's,Unknown,education,0.082650,2.638268,0.070990,0.077292,0.591972,2.622714,0.003839,0.498589
4,Bachelor's,Below Secondary,education,0.082492,2.613677,0.070654,0.076952,2.800578,2.313978,0.070878,0.473978
5,Unknown,Below Secondary,education,-0.000158,-0.024591,-0.000336,-0.000340,2.208606,-0.308735,0.067039,0.475384


In [ ]:
feature_measures.measures(tomek_df)

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Master's & above,Bachelor's,education,-0.035160,-0.510302,-0.029393,-0.033336,-1.213273,-0.376389,-0.032454,0.488074
1,Master's & above,Unknown,education,0.047239,2.249116,0.041392,0.043606,-0.639246,2.369641,-0.029149,0.489288
2,Master's & above,Below Secondary,education,0.045408,1.964929,0.039592,0.041785,1.523766,1.805427,0.037015,0.486399
3,Bachelor's,Unknown,education,0.082399,2.759417,0.070786,0.076942,0.574026,2.746029,0.003306,0.498785
4,Bachelor's,Below Secondary,education,0.080568,2.475230,0.068985,0.075121,2.737039,2.181816,0.069470,0.474494
5,Unknown,Below Secondary,education,-0.001831,-0.284187,-0.001800,-0.001821,2.163012,-0.564213,0.066164,0.475705


In [ ]:
from databalanceanalysis.databalanceanalysis.distribution_measures import DistributionBalanceMeasure
dist_measures = DistributionBalanceMeasure(cols_of_interest)
dist_measures.measures(df)


f_obs
          education  count
0        Bachelor's  33404
1   Below Secondary    458
2  Master's & above  14798
3           Unknown   2024


,feature_name,Measures.KL_DIVERGENCE,Measures.JS_DISTANCE,Measures.WS_DISTANCE,Measures.INF_NORM_DISTANCE,Measures.TOTAL_VARIANCE_DISTANCE,Measures.CHISQ_PVALUE,Measures.CHISQ
0,education,0.580926,0.401344,0.225515,0.409064,0.45103,0.0,54999.39989


In [ ]:
dist_measures.measures(smote_df)

f_obs
          education  count
0        Bachelor's  33404
1   Below Secondary  33404
2  Master's & above  33404
3           Unknown  33404


,feature_name,Measures.KL_DIVERGENCE,Measures.JS_DISTANCE,Measures.WS_DISTANCE,Measures.INF_NORM_DISTANCE,Measures.TOTAL_VARIANCE_DISTANCE,Measures.CHISQ_PVALUE,Measures.CHISQ
0,education,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
dist_measures.measures(smote_tomek_df)

f_obs
          education  count
0        Bachelor's  21817
1   Below Secondary  29441
2  Master's & above  25007
3           Unknown  22159


,feature_name,Measures.KL_DIVERGENCE,Measures.JS_DISTANCE,Measures.WS_DISTANCE,Measures.INF_NORM_DISTANCE,Measures.TOTAL_VARIANCE_DISTANCE,Measures.CHISQ_PVALUE,Measures.CHISQ
0,education,0.007521,0.043123,0.026599,0.049124,0.053198,0.0,1516.067463


In [ ]:
dist_measures.measures(tomek_df)

f_obs
          education  count
0        Bachelor's  21962
1   Below Secondary  29451
2  Master's & above  25109
3           Unknown  22258


,feature_name,Measures.KL_DIVERGENCE,Measures.JS_DISTANCE,Measures.WS_DISTANCE,Measures.INF_NORM_DISTANCE,Measures.TOTAL_VARIANCE_DISTANCE,Measures.CHISQ_PVALUE,Measures.CHISQ
0,education,0.00725,0.042344,0.026169,0.048147,0.052339,0.0,1465.850982


In [ ]:
from databalanceanalysis.databalanceanalysis.aggregate_measures import AggregateBalanceMeasure
agg_measures = AggregateBalanceMeasure(cols_of_interest)
agg_measures.measures(df)

,Measures.THEIL_L_INDEX,Measures.THEIL_T_INDEX,Measures.ATKINSON_INDEX
0,1.007477,0.580926,0.634861


In [ ]:
agg_measures.measures(smote_df)

,Measures.THEIL_L_INDEX,Measures.THEIL_T_INDEX,Measures.ATKINSON_INDEX
0,0.0,0.0,0.0


In [ ]:
agg_measures.measures(smote_tomek_df)

,Measures.THEIL_L_INDEX,Measures.THEIL_T_INDEX,Measures.ATKINSON_INDEX
0,0.007371,0.007521,0.007344


In [ ]:
agg_measures.measures(tomek_df)

,Measures.THEIL_L_INDEX,Measures.THEIL_T_INDEX,Measures.ATKINSON_INDEX
0,0.007108,0.00725,0.007083
